Import necessary libraries for program

In [53]:
import sqlite3
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import seaborn as sbn


Import World Values Survey (WVS) data csv file.

In [4]:
wvs = pd.read_csv('WVS_Wave_7.csv')

Select columns from the WVS dataset for analysis.
Print columns to ensure all are present and appear as expected.

In [5]:
wvs1 = wvs[['YEAR', 'COUNTRY', 'SECVAL', 'SKEPTICISM', 'RELATIVISM', 'DISBELIEF', 'DEFIANCE', 'EMVAL', 'AUTONOMY', 'EQUALITY', 'CHOICE', 'VOICE']]
print(wvs1.head())

   YEAR  COUNTRY    SECVAL  SKEPTICISM  RELATIVISM  DISBELIEF  DEFIANCE  \
0  2018       20  0.287062     0.09900         0.0   0.666667   0.38258   
1  2018       20  0.467525     0.65010         0.0   1.000000   0.22000   
2  2018       20  0.425304     0.37455         0.0   0.886667   0.44000   
3  2018       20  0.556170     0.65010         0.0   1.000000   0.57458   
4  2018       20  0.458949     0.37455         0.0   0.886667   0.57458   

      EMVAL  AUTONOMY  EQUALITY    CHOICE  VOICE  
0  0.413241  0.666667  0.690000  0.296296  0.000  
1  0.519722  0.000000  0.690000  0.888889  0.500  
2  0.692917  1.000000  0.690000  0.666667  0.415  
3  0.481065  0.000000  0.916667  0.592593  0.415  
4  0.475741  0.333333  0.690000  0.629630  0.250  


Filter WVS data to the year 2018.
Check for null values in the dataset.

In [6]:
wvs1_1 = wvs1[wvs1['YEAR']==2018]
wvs1_1.isnull().sum()

YEAR             0
COUNTRY          0
SECVAL         108
SKEPTICISM    1584
RELATIVISM     321
DISBELIEF      207
DEFIANCE       121
EMVAL          172
AUTONOMY       262
EQUALITY       257
CHOICE         626
VOICE          702
dtype: int64

Drop all rows with null values.
Verify null values were dropped.

In [7]:
wvs1_2 = wvs1_1.dropna()
wvs1_2.isnull().sum()

YEAR          0
COUNTRY       0
SECVAL        0
SKEPTICISM    0
RELATIVISM    0
DISBELIEF     0
DEFIANCE      0
EMVAL         0
AUTONOMY      0
EQUALITY      0
CHOICE        0
VOICE         0
dtype: int64

Create a group-by function to group the WVS individual data values into mean values per country.

In [8]:
def group_country(df, group_column, aggregate_columns, aggregation_function):
    grouped_data = df.groupby(group_column)
    aggregated_data = grouped_data[aggregate_columns].agg(aggregation_function).reset_index()
    return aggregated_data

Execute group-by function.

In [ ]:
wvs2 = group_country(wvs1_2, 'COUNTRY', ['SECVAL', 'SKEPTICISM', 'RELATIVISM', 'DISBELIEF', 'DEFIANCE', 'EMVAL', 'AUTONOMY', 
'EQUALITY', 'CHOICE', 'VOICE'], 'mean')
print(wvs2)

    COUNTRY    SECVAL  SKEPTICISM  RELATIVISM  DISBELIEF  DEFIANCE     EMVAL  \
0        20  0.413194    0.448804    0.219285   0.667209  0.317477  0.626541   
1        36  0.395258    0.339224    0.309231   0.669868  0.262710  0.635447   
2        50  0.235192    0.357076    0.364076   0.134763  0.084851  0.304735   
3        76  0.349197    0.488119    0.340013   0.305799  0.262857  0.467840   
4       152  0.453430    0.554427    0.454841   0.537442  0.267009  0.473926   
5       156  0.399067    0.250562    0.230064   0.836870  0.278773  0.392697   
6       170  0.337688    0.603700    0.321053   0.306217  0.119784  0.426456   
7       218  0.323261    0.492935    0.428717   0.293945  0.077446  0.385682   
8       276  0.384152    0.376038    0.246391   0.604469  0.309709  0.679652   
9       344  0.507181    0.412475    0.416534   0.742419  0.457297  0.497834   
10      360  0.250945    0.307360    0.374552   0.112200  0.209668  0.333719   
11      368  0.341107    0.440911    0.5

Convert Country column to string datatype to allow for storage of country names once decoded.

In [10]:
wvs2['COUNTRY'] = wvs2['COUNTRY'].astype('string')
print(wvs2.dtypes)

COUNTRY       string[python]
SECVAL               float64
SKEPTICISM           float64
RELATIVISM           float64
DISBELIEF            float64
DEFIANCE             float64
EMVAL                float64
AUTONOMY             float64
EQUALITY             float64
CHOICE               float64
VOICE                float64
dtype: object


Set Country as the index for the dataframe.

In [11]:
wvs2 = wvs2.set_index('COUNTRY')
wvs2

,SECVAL,SKEPTICISM,RELATIVISM,DISBELIEF,DEFIANCE,EMVAL,AUTONOMY,EQUALITY,CHOICE,VOICE
COUNTRY,,,,,,,,,,
20,0.413194,0.448804,0.219285,0.667209,0.317477,0.626541,0.470808,0.821144,0.716493,0.497649
36,0.395258,0.339224,0.309231,0.669868,0.262710,0.635447,0.565234,0.818340,0.693670,0.464546
50,0.235192,0.357076,0.364076,0.134763,0.084851,0.304735,0.425847,0.386657,0.107031,0.299406
76,0.349197,0.488119,0.340013,0.305799,0.262857,0.467840,0.327109,0.668469,0.395836,0.479944
152,0.453430,0.554427,0.454841,0.537442,0.267009,0.473926,0.396104,0.607169,0.461524,0.430907
156,0.399067,0.250562,0.230064,0.836870,0.278773,0.392697,0.647668,0.537589,0.203605,0.181868
170,0.337688,0.603700,0.321053,0.306217,0.119784,0.426456,0.328728,0.631724,0.300755,0.444615
218,0.323261,0.492935,0.428717,0.293945,0.077446,0.385682,0.262496,0.599698,0.279999,0.400532
276,0.384152,0.376038,0.246391,0.604469,0.309709,0.679652,0.609068,0.800839,0.657088,0.651613


Decode the country codes to replace numeric value with country name.

In [12]:
wvs2_1 = wvs2.rename(index={"20": "Andorra", "36": "Australia", "50": "Bangladesh", "76": "Brazil", "152": "Chile", "156": "China",
"170": "Colombia", "218": "Ecuador", "422": "Lebanon", "458": "Malaysia", "484": "Mexico", "566": "Nigeria", "586": "Pakistan", "604": "Peru",
"818": "Egypt", "276": "Germany", "344": "Hong Kong SAR", "360": "Indonesia", "368": "Iraq", "400": "Jordan", "398": "Kazakhstan", "630": "Puerto Rico", "642": "Romania", "410": "South Korea", "764": "Thailand",
"792": "Turkey"})
print(wvs2_1)

                 SECVAL  SKEPTICISM  RELATIVISM  DISBELIEF  DEFIANCE  \
COUNTRY                                                                
Andorra        0.413194    0.448804    0.219285   0.667209  0.317477   
Australia      0.395258    0.339224    0.309231   0.669868  0.262710   
Bangladesh     0.235192    0.357076    0.364076   0.134763  0.084851   
Brazil         0.349197    0.488119    0.340013   0.305799  0.262857   
Chile          0.453430    0.554427    0.454841   0.537442  0.267009   
China          0.399067    0.250562    0.230064   0.836870  0.278773   
Colombia       0.337688    0.603700    0.321053   0.306217  0.119784   
Ecuador        0.323261    0.492935    0.428717   0.293945  0.077446   
Germany        0.384152    0.376038    0.246391   0.604469  0.309709   
Hong Kong SAR  0.507181    0.412475    0.416534   0.742419  0.457297   
Indonesia      0.250945    0.307360    0.374552   0.112200  0.209668   
Iraq           0.341107    0.440911    0.543966   0.270702  0.10

Reset to standard index in preparation to create table in SQLite.

In [13]:
wvs3 = wvs2_1.reset_index()
wvs3

,COUNTRY,SECVAL,SKEPTICISM,RELATIVISM,DISBELIEF,DEFIANCE,EMVAL,AUTONOMY,EQUALITY,CHOICE,VOICE
0,Andorra,0.413194,0.448804,0.219285,0.667209,0.317477,0.626541,0.470808,0.821144,0.716493,0.497649
1,Australia,0.395258,0.339224,0.309231,0.669868,0.262710,0.635447,0.565234,0.818340,0.693670,0.464546
2,Bangladesh,0.235192,0.357076,0.364076,0.134763,0.084851,0.304735,0.425847,0.386657,0.107031,0.299406
3,Brazil,0.349197,0.488119,0.340013,0.305799,0.262857,0.467840,0.327109,0.668469,0.395836,0.479944
4,Chile,0.453430,0.554427,0.454841,0.537442,0.267009,0.473926,0.396104,0.607169,0.461524,0.430907
5,China,0.399067,0.250562,0.230064,0.836870,0.278773,0.392697,0.647668,0.537589,0.203605,0.181868
6,Colombia,0.337688,0.603700,0.321053,0.306217,0.119784,0.426456,0.328728,0.631724,0.300755,0.444615
7,Ecuador,0.323261,0.492935,0.428717,0.293945,0.077446,0.385682,0.262496,0.599698,0.279999,0.400532
8,Germany,0.384152,0.376038,0.246391,0.604469,0.309709,0.679652,0.609068,0.800839,0.657088,0.651613
9,Hong Kong SAR,0.507181,0.412475,0.416534,0.742419,0.457297,0.497834,0.552212,0.599864,0.424178,0.415081


Read in World Happiness Report (WHR) data csv file.
Select columns of interest for analysis.
Print columns to ensure all are present and appear as expected.

In [14]:
whr = pd.read_csv('WHR20_Data.csv')
whr1 = whr[['country', 'year', 'happiness', 'log_gdp', 'social_support', 'life_expect', 'freedom', 'generosity', 'corruption', 
'pos_affect', 'neg_affect', 'national_gov']]
print(whr1.head())

       country  year  happiness   log_gdp  social_support  life_expect  \
0  Afghanistan  2008   3.723590  7.144916        0.450662    50.799999   
1  Afghanistan  2009   4.401778  7.314788        0.552308    51.200001   
2  Afghanistan  2010   4.758381  7.421525        0.539075    51.599998   
3  Afghanistan  2011   3.831719  7.394349        0.521104    51.919998   
4  Afghanistan  2012   3.782938  7.480296        0.520637    52.240002   

    freedom  generosity  corruption  pos_affect  neg_affect  national_gov  
0  0.718114    0.178993    0.881686    0.517637    0.258195      0.612072  
1  0.678896    0.201228    0.850035    0.583926    0.237092      0.611545  
2  0.600127    0.131578    0.706766    0.618265    0.275324      0.299357  
3  0.495901    0.173452    0.731109    0.611387    0.267175      0.307386  
4  0.530935    0.246943    0.775620    0.710385    0.267919      0.435440  


Filter WHR by year 2018.
Check dataset for null values.

In [15]:
whr1_1 = whr1[whr1['year']==2018]
whr1_1.isnull().sum()

country            0
year               0
happiness          0
log_gdp            5
social_support     0
life_expect        4
freedom            1
generosity         5
corruption         8
pos_affect         2
neg_affect         2
national_gov      15
dtype: int64

Drop rows with null values.
Verify values were dropped.

In [16]:
whr1_2 = whr1_1.dropna()
whr1_2.isnull().sum()

country           0
year              0
happiness         0
log_gdp           0
social_support    0
life_expect       0
freedom           0
generosity        0
corruption        0
pos_affect        0
neg_affect        0
national_gov      0
dtype: int64

Drop the year column as it is no longer needed.
Verify column was dropped.

In [17]:
whr2 = whr1_2.drop('year', axis=1)
whr2

,country,happiness,log_gdp,social_support,life_expect,freedom,generosity,corruption,pos_affect,neg_affect,national_gov
10,Afghanistan,2.694303,7.458603,0.507516,52.599998,0.373536,-0.082319,0.927606,0.424125,0.404904,0.364666
22,Albania,5.004403,9.417863,0.683592,68.699997,0.824212,0.008337,0.899129,0.713300,0.318997,0.435338
48,Argentina,5.792797,9.813678,0.899912,68.800003,0.845895,-0.203153,0.855255,0.820310,0.320502,0.261352
62,Armenia,5.062449,9.124537,0.814449,66.900002,0.807644,-0.146650,0.676826,0.581488,0.454840,0.670828
74,Australia,7.176993,10.724133,0.940137,73.599998,0.916028,0.143235,0.404647,0.759019,0.187456,0.468837
...,...,...,...,...,...,...,...,...,...,...,...
1766,Uruguay,6.371715,9.948277,0.917316,69.000000,0.876211,-0.103306,0.682916,0.876920,0.274946,0.361706
1779,Uzbekistan,6.205460,8.738785,0.920821,65.099998,0.969898,0.316426,0.520360,0.825422,0.208660,0.969356
1819,Yemen,3.057514,7.734108,0.789422,56.700001,0.552726,-0.148377,0.792587,0.461114,0.314870,0.308151
1832,Zambia,4.041488,8.228971,0.717720,55.299999,0.790626,0.037525,0.810731,0.702698,0.350963,0.606715


Connect to newly created Happiness SQL database.
Read WHR dataframe in to create a new table in database.

In [ ]:
conn = sqlite3.connect('happiness_database.db')
table_whr = 'whr_2020'
whr2.to_sql(table_whr, conn, if_exists='replace', index=False)
conn.close()

Populate all data from WHR dataframe into the SQL table.

In [ ]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_whr}")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('Afghanistan', 2.694303274, 7.458603382, 0.507515848, 52.59999847, 0.373535544, -0.08231879, 0.927605689, 0.424125314, 0.404904395, 0.364666402)
('Albania', 5.004402637, 9.417862892, 0.683591664, 68.69999695, 0.824212313, 0.00833667, 0.899129391, 0.713299572, 0.318996727, 0.435337961)
('Argentina', 5.792796612, 9.813677788, 0.899911582, 68.80000305, 0.845894694, -0.203153208, 0.855255246, 0.820309699, 0.320502073, 0.261352271)
('Armenia', 5.062448502, 9.124536514, 0.814449012, 66.90000153, 0.807643652, -0.146650195, 0.676826358, 0.581487715, 0.454840273, 0.670827627)
('Australia', 7.17699337, 10.72413254, 0.940137267, 73.59999847, 0.916028142, 0.143235072, 0.404647499, 0.759018838, 0.187456369, 0.468836874)
('Austria', 7.396001816, 10.74663258, 0.911668122, 73.0, 0.904111981, 0.056910619, 0.52306056, 0.75235039, 0.226058796, 0.488678724)
('Azerbaijan', 5.167995453, 9.681030273, 0.781229913, 65.5, 0.772449255, -0.24819541, 0.561206281, 0.592575371, 0.191391662, 0.834371924)
('Banglades

Connect to newly created Happiness SQL database.
Read WVS dataframe in to create a new table in database.

In [20]:
conn = sqlite3.connect('happiness_database.db')
table_wvs = 'wvs_wave7'
wvs3.to_sql(table_wvs, conn, if_exists='replace', index=False)
conn.close()

Populate data from WVS dataframe into SQL table.

In [21]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute(f"SELECT * FROM {table_wvs}")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('Andorra', 0.4131937443991853, 0.4488038085539715, 0.21928475458248473, 0.6672090071283096, 0.3174773635437882, 0.6265407362525458, 0.47080786456211815, 0.821143850305499, 0.716493099796334, 0.4976485947046843)
('Australia', 0.3952580578796562, 0.3392237461318051, 0.309230547277937, 0.6698680808022923, 0.2627099283667622, 0.6354474343839541, 0.5652340446991404, 0.8183400315186247, 0.6936701891117479, 0.4645456045845272)
('Bangladesh', 0.23519153970964987, 0.35707643979504694, 0.3640762894961571, 0.13476276601195558, 0.0848507122117848, 0.30473526985482496, 0.42584678138343296, 0.38665650555081127, 0.1070309487617421, 0.2994064901793339)
('Brazil', 0.34919722215067606, 0.4881191229877656, 0.34001348486799743, 0.30579924211204124, 0.2628570965872505, 0.46783964777849324, 0.32710877591757886, 0.6684691043142306, 0.39583624404378626, 0.4799444430135222)
('Chile', 0.4534297121212121, 0.5544268344155844, 0.4548408831168831, 0.5374422673160173, 0.2670089025974026, 0.47392590043290045, 0.3961

Create new SQL table for joined WHR and WVS data.
Perform inner join on WHR and WVS data joining on the Country column as Primary Key.


In [22]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute(f"CREATE TABLE {'whr_wvs'} AS SELECT * FROM {table_wvs} INNER JOIN {table_whr} USING ('COUNTRY')")
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

OperationalError: table whr_wvs already exists

Verify all tables have been created and are present.

In [ ]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table)
conn.commit()
conn.close()

('whr_2020',)
('wvs_wave7',)
('whr_wvs',)


Export joined WHR and WVS dataset to csv  file for analysis.

In [31]:
conn = sqlite3.connect('happiness_database.db')
cursor = conn.cursor()
cursor.execute("select * from whr_wvs")
with open("whr_wvs.csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=",")
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(cursor)
conn.close()

Read in joined WHR and WVS dataset for analysis.
Verify dataframe appears as expected and all columns are present.

In [32]:
whr_wvs = pd.read_csv('whr_wvs.csv')
whr_wvs

,COUNTRY,SECVAL,SKEPTICISM,RELATIVISM,DISBELIEF,DEFIANCE,EMVAL,AUTONOMY,EQUALITY,CHOICE,...,happiness,log_gdp,social_support,life_expect,freedom,generosity,corruption,pos_affect,neg_affect,national_gov
0,Australia,0.395258,0.339224,0.309231,0.669868,0.262710,0.635447,0.565234,0.818340,0.693670,...,7.176993,10.724133,0.940137,73.599998,0.916028,0.143235,0.404647,0.759019,0.187456,0.468837
1,Bangladesh,0.235192,0.357076,0.364076,0.134763,0.084851,0.304735,0.425847,0.386657,0.107031,...,4.499217,8.263374,0.705556,64.300003,0.901471,-0.039868,0.701421,0.541345,0.361238,0.831693
2,Brazil,0.349197,0.488119,0.340013,0.305799,0.262857,0.467840,0.327109,0.668469,0.395836,...,6.190922,9.566806,0.881505,66.400002,0.750609,-0.123365,0.763251,0.749728,0.349656,0.168187
3,Chile,0.453430,0.554427,0.454841,0.537442,0.267009,0.473926,0.396104,0.607169,0.461524,...,6.436221,10.037747,0.890085,69.900002,0.788530,-0.064062,0.816297,0.832562,0.275820,0.334744
4,Colombia,0.337688,0.603700,0.321053,0.306217,0.119784,0.426456,0.328728,0.631724,0.300755,...,5.983512,9.497988,0.870970,67.699997,0.850766,-0.150524,0.854821,0.825455,0.300624,0.274870
5,Ecuador,0.323261,0.492935,0.428717,0.293945,0.077446,0.385682,0.262496,0.599698,0.279999,...,6.128010,9.250733,0.851345,68.500000,0.869364,-0.099192,0.830743,0.876475,0.328171,0.402085
6,Germany,0.384152,0.376038,0.246391,0.604469,0.309709,0.679652,0.609068,0.800839,0.657088,...,7.118364,10.735513,0.919763,72.199997,0.876888,0.035551,0.495674,0.780280,0.243215,0.592991
7,Indonesia,0.250945,0.307360,0.374552,0.112200,0.209668,0.333719,0.439794,0.388726,0.106725,...,5.340296,9.359265,0.809379,62.099998,0.879374,0.502898,0.867729,0.863718,0.295987,0.837603
8,Iraq,0.341107,0.440911,0.543966,0.270702,0.108848,0.263391,0.230460,0.366491,0.219371,...,5.099590,9.652756,0.793673,60.200001,0.643227,-0.103620,0.886390,0.659688,0.466500,0.340683
9,Kazakhstan,0.379923,0.355794,0.559509,0.343409,0.260982,0.378338,0.437381,0.482159,0.279454,...,6.007636,10.116111,0.936657,64.599998,0.840183,-0.104369,0.823783,0.693199,0.161542,0.615032


In [33]:
y = whr_wvs['happiness']
x = whr_wvs[['SECVAL', 'SKEPTICISM', 'RELATIVISM', 'DISBELIEF', 'DEFIANCE', 'EMVAL', 'AUTONOMY', 'EQUALITY', 'CHOICE', 'VOICE', 
'log_gdp', 'social_support', 'life_expect', 'freedom', 'generosity', 'corruption', 'pos_affect', 'neg_affect', 'national_gov']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [39]:
def add_constant(data_matrix):
    if isinstance(data_matrix, pd.DataFrame):
        data_matrix = data_matrix.to_numpy()

    num_rows = data_matrix.shape[0]
    constant_column = np.ones((num_rows, 1))
    data_matrix_with_constant = np.concatenate((constant_column, data_matrix), axis=1)

    return data_matrix_with_constant

Add constant to the model for the intercept.

In [43]:
x_train_c = add_constant(x_train)
x_train_c

array([[ 1.00000000e+00,  3.41106512e-01,  4.40910656e-01,
         5.43965507e-01,  2.70701927e-01,  1.08848003e-01,
         2.63390805e-01,  2.30459685e-01,  3.66491188e-01,
         2.19370808e-01,  2.37241379e-01,  9.65275574e+00,
         7.93673038e-01,  6.02000008e+01,  6.43226683e-01,
        -1.03620291e-01,  8.86389852e-01,  6.59687817e-01,
         4.66499865e-01,  3.40682983e-01],
       [ 1.00000000e+00,  3.23260755e-01,  4.92935200e-01,
         4.28716611e-01,  2.93945284e-01,  7.74459889e-02,
         3.85681678e-01,  2.62496373e-01,  5.99698440e-01,
         2.79999308e-01,  4.00532476e-01,  9.25073338e+00,
         8.51345003e-01,  6.85000000e+01,  8.69363725e-01,
        -9.91920530e-02,  8.30743194e-01,  8.76474917e-01,
         3.28171045e-01,  4.02085185e-01],
       [ 1.00000000e+00,  3.66736018e-01,  3.95160408e-01,
         5.76491199e-01,  3.42883256e-01,  1.52409254e-01,
         4.12287506e-01,  5.22492309e-01,  5.51702033e-01,
         2.08731712e-01,  3.6

In [44]:
model = LinearRegression()
model.fit(x_train_c, y_train)

LinearRegression()

In [45]:
print("Coefficients:", model.coef_)

Coefficients: [-1.64729341e-14  4.85095118e-01  8.74854716e-02  3.47638992e-01
  1.41117589e+00  9.40775630e-02 -3.50480290e-01 -2.38941943e+00
  1.60617468e+00 -5.90477593e-01 -2.61665915e-02  2.47952903e-02
  2.09243860e+00  3.42269645e-03 -3.31977514e-01  8.40103236e-01
 -3.23478567e+00  1.18028986e+00 -2.51912367e+00 -3.98221248e-01]


In [46]:
print("Intercept:", model.intercept_)

Intercept: 6.098249880671941


In [ ]:
y_pred = model.predict(x_train_c)
print("R-squared:", r2_score(y_test, y_pred))

ImportError: cannot import name 'r2_score' from 'sklearn' (c:\Users\spaul\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\__init__.py)